In [1]:
### 讀檔
TotalQueryPath = 'C:/Users/dnc22/IR_HW2/query_list.txt'
TotalDocumentPath = 'C:/Users/dnc22/IR_HW2/doc_list.txt'
QuerysPath = 'C:/Users/dnc22/IR_HW2/queries/'
DocumentsPath = 'C:/Users/dnc22/IR_HW2/docs/'

# Document_dict =
# {
#     'FBIS3-10082' = ['languag','f','p','105','spanish' ...]
#     ...
# }

# Document_Count = 
# {
#     dict:'FBIS3-10082' = 
#     {
#         'languag' = 2,
#         'f' = 1,
#         'p' = 1,
#         ...
#         'de' = 6,
#         ...
#     }
# }

# Variable_AvgWord          5000篇 Document 的平均字數

Query_dict = {}
Document_Count = {}
Variable_AvgWord = 0

QuerySet = []

with open (TotalQueryPath, 'r') as f:
    for QueryFileName in f.readlines():
        QueryFileName = QueryFileName.strip('\n')

        with open(QuerysPath + QueryFileName + '.txt', 'r') as qf:
            query_list = []
            for query_read in qf.readlines():
                for word in query_read.split(' '):
                    word = word.lower()
                    query_list.append(word)
                    QuerySet.append(word)

            #print(QueryFileName, query_list)
            Query_dict[QueryFileName] = query_list
#print(Query_dict)
#print(QuerySet)

Document_dict = {}
with open (TotalDocumentPath, 'r') as f:
    for DocumentFileName in f.readlines():
        DocumentFileName = DocumentFileName.strip('\n')

        with open(DocumentsPath + DocumentFileName + '.txt', 'r') as df:
            document_list = []
            for document_read in df.readlines():
                for word in document_read.split(' '):
                    word = word.lower()
                    if word in QuerySet:
                        document_list.append(word)

            #print(QueryFileName, query_list)
            dcount = {}
            Variable_AvgWord += len(document_list)

            for word in document_list:
                dcount[word] = dcount.get(word,0) + 1

            Document_dict[DocumentFileName] = document_list
            Document_Count[DocumentFileName] = dcount

Variable_AvgWord /= len(Document_dict)

#print(Document_dict)
#print(Document_Count)
#print(Variable_AvgWord)


In [2]:
### 計算 TF
import math

# Query_TF = 
# {
#     '301' = 
#     {
#         'intern' = 0.33333 ,
#         'organ' = 0.33333 ,
#         'crime' = 0.33333 
#     }
#     ...
# }

# Word_Count =                              # 計算Query出現在全部的 Document 幾次(一篇最多一次), 用來計算IDF
# {
#     'intern': 52次
#     'organ': 38次
#     ...
# }

Query_TF = {}
Word_Count = {}
for QueryDict in Query_dict:
    FileName = QueryDict
    #print(FileName, len(Query_dict[FileName]))
    qdict = {}
    for word in Query_dict[FileName]:       # 先計算出現幾次
        if word not in qdict:
            qdict[word] = 1
            Word_Count[word] = 0            # Query重複出現, 不重複計算
        else:
            qdict[word] += 1
    
    for word in qdict:                      # 最後直接一次除以整篇文章出現的次數去算頻率
        #qdict[word] = qdict[word] / len(Query_dict[FileName])
        qdict[word] = qdict[word]

    Query_TF[FileName] = qdict

#print(Query_TF)

# document TF
Document_TF = {}
for DocumentDict in Document_dict:
    FileName = DocumentDict
    ddict = {}
    wordSet = []        

    for word in Document_dict[FileName]:
        if word not in ddict:
            ddict[word] = 1
        else:
            ddict[word] += 1
        
        if word not in wordSet:
            if word in Word_Count:
                Word_Count[word] += 1           # 計算 query 在這篇 document 有沒有出現過
                                                # 條件: 必須是 query 且 是這篇 document 第一次出現
        wordSet.append(word)

    for word in ddict:
        #ddict[word] = ddict[word] / len(Document_dict[FileName])
        ddict[word] = ddict[word]
    #print(FileName, len(Document_dict[FileName]))
    Document_TF[FileName] = ddict

#print(Document_TF)
#print(Word_Count)

In [3]:
### 計算IDF

# Query_IDF = 
# {
#     '301' = 
#     {
#         'intern' = 0.84728 ,
#         'organ' = 2.17928 ,
#         'crime' = 3.44979
#     }
#     ...
# }

Query_IDF = {}
TotalCount = len(Query_dict) + len(Document_dict)
#print(TotalCount)

for word in Word_Count:
    numerator = TotalCount - Word_Count[word] + 0.5
    denominator = Word_Count[word] + 0.5
    idf = numerator / denominator
    #print(word,idf)
    Query_IDF[word] = math.log10(idf + 1)

#print(Query_IDF)

In [19]:
### 計算 BM25 分數

# 參數
Variable_K1 = 0.01
Variable_K3 = 1
Variable_B = 0.0

Score_list = {}
# Score_list = 
# {
#     dict '301' = 
#     {
#         'FBIS3-10082' = 0.12345 ,
#         'FBIS3-10083' = 0.23456 ,
#         ...
#     }
#     dict '302' = 
#     {
#         'FBIS3-10082' = 0.54321 ,
#         'FBIS3-10083' = 0.23456 ,
#         ...
#     }
# }

for queryfile in Query_dict:
    querylist = Query_dict[queryfile]
#   queryfile = '301'    
#   querylist = ['intern', 'organ', 'crime']
    score_dict = {}

    for docfile in Document_dict:
        docwordLen = len(Document_dict[docfile])
#       docfile = 'FBIS3-10082'
#       Document_Count['FBIS3-10082']['intern'] = 1
#       docwordLen = 字節長度
        score = 0

        for term in querylist:
            idf = Query_IDF[term]
            count = 0
            if term in Document_Count[docfile]:
                count = Document_Count[docfile][term]

            if docwordLen == 0:
                docwordLen = 1
            tf = count / docwordLen
            numerator = idf * tf * (Variable_K1 + 1)
            denominator = tf + Variable_K1 * ( 1 - Variable_B + Variable_B * (docwordLen / Variable_AvgWord))
            K3 = ((Variable_K3 + 1) * Query_TF[queryfile][term] ) / (Variable_K3 + Query_TF[queryfile][term])

            score += (numerator / denominator) * K3
        score_dict[docfile] = score
        #print(queryfile, docfile, score)
    #print(queryfile, score_dict)
    Score_list[queryfile] = score_dict
#print(Score_list)

In [20]:
### 輸出

outPath = 'C:/Users/dnc22/IR_HW2/output.txt'

with open(outPath, 'w') as f:
    f.writelines('Query,RetrievedDocuments\n')

    for querydict in Score_list:
        #print(Score_list[querydict])
        doc_score = sorted(Score_list[querydict].items(), key=lambda item: item[1] , reverse=True)
        #print(querydict , doc_score)
        f.writelines(querydict + ',')
        for docFileName in doc_score:
            f.writelines(docFileName[0])
            if docFileName != doc_score[-1]:
                f.writelines(' ')
        f.writelines('\n')